In [3]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from time import sleep
import pandas as pd
import re

In [ ]:
df = pd.read_csv("links.csv")
df

In [5]:
def sanitize_filename(caption):
    caption = re.sub(r'Duration:\s*\d{1,2}:\d{2}\s*minutes$', '', caption, flags=re.MULTILINE)
    filename = re.sub(r'[^\w\s-]', '', caption)
    filename = re.sub(r'[\s\n]+', '_', filename)
    filename = filename.strip('_')
    return filename[:255]

In [19]:
def save_download_video(youtube_link):
    try:
        url = 'https://en.y2mate.is/v96/'

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

        driver.get(url)

        driver.maximize_window()

        a = driver.find_elements(By.ID, "txtUrl")[0].send_keys(youtube_link)

        b = driver.find_elements(By.ID, "btnSubmit")[0].click()

        sleep(3)

        trs = driver.find_elements(By.TAG_NAME, 'tr')

        for i in trs:
            if "360p" in i.text:
                tr = i
                break

        tr.find_element(By.TAG_NAME, 'button').click()

        sleep(3)

        while True:

            try:
                download_btn = driver.find_element(By.ID, "btnOk")
                break

            except:
                sleep(3)
                continue

        x = download_btn.find_element(By.TAG_NAME, "button").find_element(By.TAG_NAME, "a")

        sleep(3)

        download_link = x.get_attribute("href")

        caption = driver.find_element(By.TAG_NAME, "b").text

        caption = sanitize_filename(caption)

        return download_link, caption
    
    except Exception as e:
        print(e)
        return None, None

In [ ]:
for index, row in df.iterrows():

    download_link, caption = save_download_video(row['youtube_link'])

    df.at[index, 'download_link'] = download_link
    df.at[index, 'caption'] = caption
    df.at[index, 'success'] = True

    df.to_csv("links.csv", index=False)